In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
  plt.show()

In [3]:
dataset="D:\dataset\final\cleandata"

In [4]:
Corpus = pd.read_csv(r"D:\dataset\final\cleandata.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.3)

In [6]:

print(len(Train_X))
print(len(Train_Y))
print(len(Test_X))
print(len(Test_Y))

1244422
1244422
533325
533325


In [7]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [13]:
vocab_size = 120000
embedding_dim = 100
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(Train_X)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(Train_X)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(Test_X)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [14]:

embeddings_index = {};
with open('D:/dataset/final/glove.6B.100d.txt',encoding="utf8") as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 100)          12000100  
_________________________________________________________________
dropout (Dropout)            (None, 120, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 116, 64)           32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 29, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 12,065,253
Tra

In [17]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final),verbose=2)

Train on 1244422 samples, validate on 533325 samples
Epoch 1/10
1244422/1244422 - 1243s - loss: 0.0112 - acc: 0.9961 - val_loss: 0.0072 - val_acc: 0.9975
Epoch 2/10
1244422/1244422 - 1230s - loss: 0.0072 - acc: 0.9974 - val_loss: 0.0060 - val_acc: 0.9978
Epoch 3/10
1244422/1244422 - 1223s - loss: 0.0063 - acc: 0.9978 - val_loss: 0.0054 - val_acc: 0.9981
Epoch 4/10
1244422/1244422 - 1270s - loss: 0.0058 - acc: 0.9979 - val_loss: 0.0056 - val_acc: 0.9980
Epoch 5/10
1244422/1244422 - 1268s - loss: 0.0054 - acc: 0.9981 - val_loss: 0.0051 - val_acc: 0.9982
Epoch 6/10
1244422/1244422 - 1292s - loss: 0.0051 - acc: 0.9982 - val_loss: 0.0050 - val_acc: 0.9982
Epoch 7/10
1244422/1244422 - 1318s - loss: 0.0050 - acc: 0.9982 - val_loss: 0.0051 - val_acc: 0.9983
Epoch 8/10
1244422/1244422 - 1264s - loss: 0.0048 - acc: 0.9983 - val_loss: 0.0049 - val_acc: 0.9982
Epoch 9/10
1244422/1244422 - 1213s - loss: 0.0046 - acc: 0.9983 - val_loss: 0.0051 - val_acc: 0.9982
Epoch 10/10
1244422/1244422 - 1187s - 

In [18]:
#save model 
model.save('LSTMmodel1')

In [2]:
import pickle 
f = open('D:/NLP project/models/tokenizer1.pickle', 'rb')
tokenizer = pickle.load(f)
model =tf.keras.models.load_model('D:/NLP project/models/LSTMmodel.h5')
text=["MILF"]
sequences = tokenizer.texts_to_sequences(text)
padded = pad_sequences(sequences, maxlen=120, padding='post', truncating='post')
x=model.predict(padded)
print(x)
if x>0.5:
    print("inapprpriate context")
else:
    print("appropriate context")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


KeyError: 'sample_weight_mode'

In [25]:
plot_graphs(history, metric)

NameError: name 'history' is not defined

In [31]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_sentence(padded[0])+"\n")
print(Train_X[2])

ahmed ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?



KeyError: 2

In [32]:
#embedding projector 

import io
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

out_v = io.open('vecs1.tsv', 'w', encoding='utf-8')
out_m = io.open('meta1.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    try:
        word = reverse_word_index[word_num]
        embeddings = weights[word_num]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    except KeyError:
        pass
out_v.close()
out_m.close()

(120001, 100)


In [3]:
tf --version


NameError: name 'version' is not defined